# Proj II: Implementing Shor's Factorization Algorithm in Python and Qiskit

- This project centers on the practical implementation of Shor's factorization algorithm using Python and Qiskit.
- Shor's algorithm is a significant breakthrough in quantum computing and holds the promise of efficiently factoring large integers, a task that poses substantial challenges for classical computers.
- Your objective will be to translate this algorithm into a functional program, enabling the factorization of composite numbers.

- The implementation will involve using both classical and quantum components, providing you with a valuable opportunity to explore the synergy between the two computing paradigms.
- This project offers a pragmatic approach to understanding its real-world applications.
- It is a chance to grasp the algorithmic essence of quantum factorization and the complexities it can address in practice.

In [12]:
from math import floor, log2, pi

from qiskit import *
from qiskit.quantum_info import Statevector
from qiskit.circuit import Qubit

from my_quantum_circuit import MyQuantumCircuit

In [ ]:
"""Paste your IBMQ API token here:"""
IBMQ_API_TOKEN: str = ''
IBMQ.save_account(IBMQ_API_TOKEN)
IBMQ.load_account()
pass

In [48]:
"""Helper functions"""

def gcd(a: int, b: int) -> int:
    """
    Euclid's original version of the greates common divisor algorithm, using the subtraction method.
    :param a: first input number,
    :type a: positive integer,
    :param b: second input number,
    :type b: positive integer,
    :return: the greatest common divisor of a and b.
    """
    assert a, b > 0
    while a != b:
        if a > b:
            a = a - b
        else:
            b = b - a
    return a

def get_min_n_bits(n) -> int:
    """
    Get the minimum number of bits needed to represent a given number in binary form.
    :param n: the number
    :type n: positive integer
    :return: the minimum number of bits for n's binary representation
    """
    assert n >= 0
    return floor(log2(n)) + 1

def extended_euclid(a: int, n: int) -> int:
    """
    Extended Euclidean algorithm.
    Computes the inverse of `a` in the group (R_n, *). I.e., a^(-1) such that `(a * a^(-1)) mod n = 1 mod n`.
    """

    assert 0 < a < n and gcd(a, n) == 1, 'a must be in the set R_n'

    t, new_t = 0, 1
    r, new_r = n, a

    while new_r != 0:
        quotient = r // new_r
        t, new_t = new_t, t - quotient * new_t
        r, new_r = new_r, r - quotient * new_r

    if r > 1:
        raise Exception('a is not invertible')
    if t < 0:
        t = t + n
    return t

def sieve_of_eratosthenes(n: int) -> set[int]:
    prime = [True for i in range(n+1)]
    p = 2
    while p * p <= n:
        if prime[p]:
            for i in range(p * p, n+1, p):
                prime[i] = False
        p += 1

    primes_set = set()
    for p in range(2, n+1):
        if prime[p]:
            primes_set.add(p)
    return primes_set

In [49]:
"""Class for Shor's algorithm quantum circuit"""

class ShorQuantumCircuit(MyQuantumCircuit):

    def __init__(self, circuit=None):
        super().__init__(circuit)

    # -----------------------------------------------------------------------

    @staticmethod
    def shor_algorithm(n: int, n_precision_bits: int, n_shots: int,) -> (int, int):
        """
        Shor's factorization algorithm
        :param n: the integer to factorize
        :param n_precision_bits: degree of precision of the quantum component (proportional to the likelihood of getting the correct answer)
        :param n_shots: number of times to execute the quantum circuit
        :return: factors of n
        """

        """Precondition(s)"""
        assert n > 1, 'Number to factorize must be n > 1.'

        """1. If n is even, return {2, n / 2} as its factors."""
        if not(n & 1):
            return 2, n // 2

        """2. Check if n is of the form `n = p ^ q`, where p and q are positive integers, with p being prime."""
        """Firstly, check if n is of the form `n = p ^ 1`, where p is a prime integer."""
        if n in sieve_of_eratosthenes(n):
             return 1, n
        """Secondly, check if n is of the form `n = p ^ q` where q > 1."""
        for q in reversed(range(2, floor(log2(n)) + 1)):
            p = n ** (1 / q)
            if p == floor(p):
                p = floor(p)
                return p, n // p

        """3. Pick a number `a` from the set of least-residues-system `Z_n` (i.e., 1 < a < n)."""
        for a in range(2, n):
            """4. Let `k = gcd(a, n)`.
                If `k != 1`, then return {k, n / k} as the non-trivial factors of `n`."""
            k = gcd(a, n)
            if k != 1:
                return k, n // k

            """5. Use the quantum subroutine to find the order `r` of `a` `modulo n`."""
            r = ShorQuantumCircuit._shor_quantum_subroutine(n, n_precision_bits, a, n_shots)

            """6. If `r` is odd, go to to step 3.
                Else continue with step 7."""
            if not(r & 1):
                """7. Compute both `gcd(n, a ** (r // 2) - 1)` and `gcd(n, a ** (r // 2) - 1)`.
                    Return whichever is non-trivial.
                    Else go back to step 3."""
                d = gcd(n, a ** (r // 2) - 1)
                if d != 1 and d != n:
                    return d, n // d

                d = gcd(n, a ** (r // 2) + 1)
                if d != 1 and d != n:
                    return d, n // d

        raise Exception('Could not find factors.')

    # -----------------------------------------------------------------------

    @staticmethod
    def _shor_quantum_subroutine(n: int, n_precision_bits: int, a: int, n_shots: int):
        """
        Quantum subroutine of Shor's factorization algorithm.
        Computes the order r (a.k.a. repeat period) of a a w.r.t. the number to factorize n:
            `(a ^ r) mod n = a ^ 0 = 1`.
        :param n: the number to factorize
        :param n_precision_bits: degree of precision of the quantum component
        :param a: current number coprime with the number to factorize
        :param n_shots: number of times to run the quantum circuit
        :return: order (i.e., repeat period) of the coprime number
        """

        """Compute the number of qubits for the work register (i.e., the minimum number of bits for the binary string representation of n);"""
        n_work_qubits = get_min_n_bits(n)
        """Initialize precision register (used as ancilla register in the Shor Quantum-Phase-Estimation);"""
        precision_qreg = QuantumRegister(n_precision_bits)
        precision_creg = ClassicalRegister(n_precision_bits)
        """Initialize work register (used as encoding register in the Shor Quantum-Phase-Estimation);"""
        work_qreg = QuantumRegister(n_work_qubits)

        """Create the circuit for the quantum subroutine;"""
        shor_circuit = ShorQuantumCircuit(QuantumCircuit(precision_qreg, work_qreg, precision_creg))
        """Apply Quantum-Phase-Estimation adapted to the quantum subroutine for Shor's algorithm;"""
        shor_circuit.shor_quantum_phase_estimation(precision_qreg, work_qreg, a, n)
        """Measure, execute and return the repeat_period;"""
        return shor_circuit\
            .measure_qreg_to_creg(precision_qreg, precision_creg)\
            .execute_circuit(n_shots)\
            .get_shor_repeat_period()

    # -----------------------------------------------------------------------

    def shor_quantum_phase_estimation(self, precision_qubits, work_qubits, x: int, n: int):

        n_work_qubits: int = len(work_qubits)
        n_precision_qubits: int = len(precision_qubits)

        """Create superposition in precision qubits (precision qubits are initialized to |0⟩ by default);"""
        self.had(*precision_qubits)
        """Initialize work qubits to |1⟩"""
        self.initialize_qreg(work_qubits, 1)

        """Create helper registers"""
        auxilliary_qreg = QuantumRegister(n_work_qubits, 'Auxilliary')
        overflow_qreg = QuantumRegister(1, 'Overflow')
        ancilla_qreg = QuantumRegister(1, 'Ancillary')
        """Add helper registers to circuit"""
        self.circuit.add_register(auxilliary_qreg, overflow_qreg, ancilla_qreg)
        """Extend auxilliary register with an overflow qubit for the add_x_mod_n operation"""
        auxilliary_qubits_overflow_extension = auxilliary_qreg[:] + overflow_qreg[:]

        """Quantum-Phase-Estimation -> Apply Controlled-U_a gates"""
        for precision_bit_idx in range(n_precision_qubits):
            self.c_mult_x_mod_n(
                precision_qubits[precision_bit_idx],
                work_qubits,
                auxilliary_qubits_overflow_extension,
                (x ** (2 ** precision_bit_idx)) % n, n,
                ancilla_qreg[:]
            )

        self.inverse_quantum_fourier_transform(precision_qubits)

        return self

    # -----------------------------------------------------------------------

    def had(self, *qubit):
        """HAD gate"""
        self.circuit.h(qubit)
        return self

    def x(self, *qubit):
        """NOT gate"""
        self.circuit.x(qubit)
        return self

    def c_not(self, control_qubit, target_qubit):
        """Controlled-NOT gate"""
        self.circuit.cx(control_qubit, target_qubit)
        return self

    def cc_not(self, control_qubit_1, control_qubit_2, target_qubit):
        """Controlled-Controlled-NOT gate"""
        self.circuit.ccx(control_qubit_1, control_qubit_2, target_qubit)
        return self

    # -----------------------------------------------------------------------

    def phase(self, theta, qubit):
        """PHASE gate"""
        self.circuit.p(theta, qubit)
        return self

    def c_phase(self, theta, control_qubit, target_qubit):
        """Controlled-PHASE gate."""
        self.circuit.cp(theta, control_qubit, target_qubit)
        return self

    def cc_phase(self, theta, control_qubit_1, control_qubit_2, target_qubit):
        """Controlled-Controlled-PHASE gate."""
        self.c_phase(theta/2, control_qubit_1, target_qubit)\
            .c_not(control_qubit_2, control_qubit_1)\
            .c_phase(-theta/2, control_qubit_1, target_qubit)\
            .c_not(control_qubit_2, control_qubit_1)\
            .c_phase(theta/2, control_qubit_2, target_qubit)
        return self

    # -----------------------------------------------------------------------

    def swap(self, qubit1, qubit2):
        """SWAP gate"""
        self.circuit.swap(qubit1, qubit2)
        return self

    def cswap(self, control_qubit, target_qubit1, target_qubit2):
        """Controlled-SWAP gate"""
        self.circuit.cswap(control_qubit, target_qubit1, target_qubit2)
        return self

    # -----------------------------------------------------------------------

    def phi_add(self, x, qft_qubits):
        """
        Phi-Addition of a classical integer to a QFT-ed quantum register.
        Remember that the `qubits` parameter must have an additional overflow qubit at its end.
        :param x: first term
        :type x: classical integer
        :param qft_qubits: QFT-ed second term
        :type qft_qubits: QFT-ed list[Qubit] or QFT-ed QuantumRegister
        :return: self
        """
        n_qubits = len(qft_qubits)
        n_states = 2 ** n_qubits
        assert x <= n_states

        # perform 'wave interference':
        unit_angle = 2 * pi * (x / n_states)  # similar to the angular frequency formula: omega = f * (2 * pi)
        for i in range(n_qubits):
            self.phase(theta=-(2 ** i) * unit_angle, qubit=qft_qubits[i])
        return self

    def c_phi_add(self, x, control_qubit, target_qft_qubits):
        """
        Controlled-Phi-Addition of a classical integer to a QFT-ed quantum register.
        Remember that the `qubits` parameter must have an additional overflow qubit at its end.
        :param x: first term
        :type x: classical integer
        :param control_qubit: control qubit
        :type control_qubit: Qubit
        :param target_qft_qubits: QFT-ed second term
        :type target_qft_qubits: QFT-ed list[Qubit] or QFT-ed QuantumRegister
        :return: self
        """
        n_qubits = len(target_qft_qubits)
        n_states = 2 ** n_qubits
        assert x <= n_states

        unit_angle = 2 * pi * (x / n_states)
        for i in range(n_qubits):
            self.c_phase(
                theta=-(2 ** i) * unit_angle,
                control_qubit=control_qubit,
                target_qubit=target_qft_qubits[i]
            )
        return self

    def cc_phi_add(self, x, control_qubit_1, control_qubit_2, target_qft_qubits):
        """
        Controlled-Controlled-Phi-Addition of a classical integer to a QFT-ed quantum register.
        Remember that the `qubits` parameter must have an additional overflow qubit at its end.
        :param x: first term
        :type x: classical integer
        :param control_qubit_1: first control qubit
        :type control_qubit_1: Qubit
        :param control_qubit_2: second control qubit
        :type control_qubit_2: Qubit
        :param target_qft_qubits: QFT-ed second term
        :type target_qft_qubits: QFT-ed list[Qubit] or QFT-ed QuantumRegister
        :return: self
        """
        n_qubits = len(target_qft_qubits)
        n_states = 2 ** n_qubits
        assert x <= n_states

        unit_angle = 2 * pi * (x / n_states)
        for i in range(n_qubits):
            self.cc_phase(
                theta=-(2 ** i) * unit_angle,
                control_qubit_1=control_qubit_1,
                control_qubit_2=control_qubit_2,
                target_qubit=target_qft_qubits[i]
            )
        return self

    # -----------------------------------------------------------------------
    """phi_add_x_mod_n IS NOT NEEDED FOR THE ACTUAL ALGORITHM"""
    def phi_add_x_mod_n(self, qft_qubits_overflow_extension, x, n, ancilla_qubit, reverse=False):
        """
        Implements `+_n` operator of the abelian group (Z_n, +_n), where `+_n: qubits := (qubits + x) % n`.
        :param qft_qubits_overflow_extension: first term of addition, and modulo dividend (must have an additional overflow qubit)
        :type qft_qubits_overflow_extension: list[Qubit] or QuantumRegister
        :param x: second term of addition
        :type x: integer
        :param n: modulo divisor
        :type n: integer
        :param ancilla_qubit: helps compute the `mod n`; ancilla must be |0⟩ before and after `phi_add_mod`
        :type ancilla_qubit: Qubit
        :param reverse: True if the reversed phi_add_x_mod_n needs to be applied (computes the inverse of x and adds it)
        :type reverse: bool
        :return: self
        """

        """Pre-condition: x and n must not have more bits than the number of qubits (minus the overflow qubit)."""
        n_qubits = len(qft_qubits_overflow_extension) - 1
        assert get_min_n_bits(n) <= n_qubits
        assert get_min_n_bits(x) <= n_qubits
        """Pre-condition: x in Z_n"""
        assert 0 <= x < n, f'x must be in Z_n, (x={x}, n={n})'

        """If the reverse of this gate is applied, then add the inverse of x in the group (Z_n, +_n)."""
        x = (-x % n) if reverse else x
        """The qubits have an additional overflow qubit (needed for the modulo operator)."""
        overflow_qubit = qft_qubits_overflow_extension[-1]
        """Add x;"""
        self.phi_add(x, qft_qubits_overflow_extension)
        """Subtract n (helps detect whether overflow occured);"""
        self.phi_add(-n, qft_qubits_overflow_extension)
        """Determine if overflow occured :
            - overflow qubit is 1 if no overflow, and 0 if overflow,
            - same for ancilla;"""
        self\
            .inverse_quantum_fourier_transform(qft_qubits_overflow_extension)\
            .c_not(overflow_qubit, ancilla_qubit)\
            .quantum_fourier_transform(qft_qubits_overflow_extension)
        """If overflow did not occur (i.e., ancilla is 1), add n back;"""
        self.c_phi_add(n, ancilla_qubit, qft_qubits_overflow_extension)
        """Reset ancilla qubit to 0:
            - (a + b) mod N ≥ a <=> a + b < N,
            - compare `(qubits + x) mod n` with the value x using essentially the same trick as before;"""
        self\
            .phi_add(-x, qft_qubits_overflow_extension)\
            .inverse_quantum_fourier_transform(qft_qubits_overflow_extension)\
            .x(overflow_qubit)\
            .c_not(overflow_qubit, ancilla_qubit)\
            .x(overflow_qubit)\
            .quantum_fourier_transform(qft_qubits_overflow_extension)\
            .phi_add(x, qft_qubits_overflow_extension)
        return self

    def cc_phi_add_x_mod_n(
            self,
            control_qubit_1,
            control_qubit_2,
            qft_qubits_overflow_extension,
            x, n,
            ancilla_qubit,
            reverse=False
    ):
        """
        Implements  the doubly-controlled `+_n` operator of the abelian group (Z_n, +_n), where `+_n: qubits := (qubits + x) % n`.
        :param control_qubit_1: first control qubit
        :type control_qubit_1: Qubit
        :param control_qubit_2: second control qubit
        :type control_qubit_2: Qubit
        :param qft_qubits_overflow_extension: first term of addition, and modulo dividend (must have an additional overflow qubit)
        :type qft_qubits_overflow_extension: list[Qubit] or QuantumRegister
        :param x: second term of addition
        :type x: integer
        :param n: modulo divisor
        :type n: integer
        :param ancilla_qubit: helps compute the `mod n`; ancilla must be |0⟩ before and after `phi_add_mod`
        :type ancilla_qubit: Qubit
        :param reverse: True if the reversed phi_add_x_mod_n needs to be applied (computes the inverse of x and adds it)
        :type reverse: bool
        :return: self
        """

        """Pre-condition: x and n must not have more bits than the number of qubits (minus the overflow qubit)."""
        n_qubits = len(qft_qubits_overflow_extension) - 1
        assert get_min_n_bits(n) <= n_qubits
        assert get_min_n_bits(abs(x)) <= n_qubits
        """Pre-condition: x in Z_n"""
        assert 0 <= x < n, f'x must be in Z_n, (x={x}, n={n})'

        """If the reverse of this gate is applied, then add the inverse of x in the group (Z_n, +_n)."""
        x = (-x % n) if reverse else x
        """The qubits have an additional overflow qubit (needed for the modulo operator)."""
        overflow_qubit = qft_qubits_overflow_extension[-1]
        """Add x;"""
        self.cc_phi_add(x, control_qubit_1, control_qubit_2, qft_qubits_overflow_extension)
        """Subtract n (helps detect whether overflow occured);"""
        self.phi_add(-n, qft_qubits_overflow_extension)
        """Determine if overflow occured :
            - overflow qubit is 1 if no overflow, and 0 if overflow,
            - same for ancilla;"""
        self\
            .inverse_quantum_fourier_transform(qft_qubits_overflow_extension)\
            .c_not(overflow_qubit, ancilla_qubit)\
            .quantum_fourier_transform(qft_qubits_overflow_extension)
        """If overflow did not occur (i.e., ancilla is 1), add n back;"""
        self.c_phi_add(n, ancilla_qubit, qft_qubits_overflow_extension)
        """Reset ancilla qubit to 0;"""
        self\
            .cc_phi_add(-x, control_qubit_1, control_qubit_2, qft_qubits_overflow_extension)\
            .inverse_quantum_fourier_transform(qft_qubits_overflow_extension)\
            .x(overflow_qubit)\
            .c_not(overflow_qubit, ancilla_qubit)\
            .x(overflow_qubit)\
            .quantum_fourier_transform(qft_qubits_overflow_extension)\
            .cc_phi_add(x, control_qubit_1, control_qubit_2, qft_qubits_overflow_extension)
        return self

    # -----------------------------------------------------------------------

    def _c_mult_x_mod_n_helper(
            self,
            control_qubit,
            qubits,
            auxilliary_qubits_overflow_extension,  # auxilliary register, must be 0 initially
            x, n,
            ancilla_qubit,
            reverse=False,
    ):
        """
        Helper method for the c_mult_x_mod_n operator.
        Implements: _c_mult_x_mod_n_helper: R_n -> R_n, aux_qubits := aux_qubits + (qubits * x) % n
        """

        self.quantum_fourier_transform(auxilliary_qubits_overflow_extension)
        for idx in range(len(qubits)):
            self.cc_phi_add_x_mod_n(
                control_qubit_1=control_qubit,
                control_qubit_2=qubits[idx],
                qft_qubits_overflow_extension=auxilliary_qubits_overflow_extension,
                x=((2 ** idx) * x) % n, n=n,  # ((2 ** idx) * x) % n because (a + b) mod n == (a mod n + b mod n) mod n
                ancilla_qubit=ancilla_qubit,
                reverse=reverse
            )
        self.inverse_quantum_fourier_transform(auxilliary_qubits_overflow_extension)
        return self

    # -----------------------------------------------------------------------

    def c_mult_x_mod_n(
            self,
            control_qubit,
            qubits,
            auxilliary_qubits_overflow_extension,
            x, n,
            ancilla_qubit,
            reverse=False
    ):  # aka. controlled-U_a gate
        """
        Implements the controlled `*_n` operator of the abelian group (R_n, *_n), where `*_n`: qubits := (qubits * x) mod n,
            and `qubits` is a quantum register containing an integer, and x and n are classical integers.
        :param control_qubit: control qubit
        :type control_qubit: Qubit
        :param qubits: qubits containing an integer, on which to apply and store the result of `*_n`
        :type qubits: list[Qubit] or QuantumRegister
        :param auxilliary_qubits_overflow_extension: helper qubits (extended with an overflow qubit), on which `_c_mult_x_mod_n_helper` is applied, must be |0⟩ before and after
        :type auxilliary_qubits_overflow_extension: list[Qubit] or QuantumRegister
        :param x: second term of addition
        :type x: classical integer
        :param n: modulo divisor
        :type n: classical integer
        :param ancilla_qubit: helper qubit, used for detecting overflows (and implicitly for computing `mod n`)
        :type ancilla_qubit: Qubit
        :return: self
        """
        """Pre-condition(s):"""
        assert len(qubits) == len(auxilliary_qubits_overflow_extension) - 1, 'Lists of qubits and auxilliary qubits must have a difference of 1 in sizes.'
        assert 0 < x < n and gcd(x, n) == 1, f'x must be in R_n, (x={x}, n={n})'

        """To apply the reverse of this gate, compute the inverse of x in the group (R_n, *_n) and apply `*_n`."""
        x = extended_euclid(x, n) if reverse else x

        """Compute `*_n` and store it in `auxilliary_qubits_overflow_extension`;"""
        self._c_mult_x_mod_n_helper(
            control_qubit=control_qubit,
            qubits=qubits,
            auxilliary_qubits_overflow_extension=auxilliary_qubits_overflow_extension,
            x=x, n=n,
            ancilla_qubit=ancilla_qubit,
        )

        """Swap `qubits` with the `*_n` result from `auxilliary_qubits_overflow_extension`;"""
        for idx in range(len(qubits)):
            self.cswap(control_qubit, qubits[idx], auxilliary_qubits_overflow_extension[idx])

        """Compute the inverse of x in the group (R_n, *_n) using the Extended Euclidean algorithm;"""
        x_inverse = extended_euclid(x, n)

        """Invert the `*` operation on `auxilliary_qubits_overflow_extension`, to restore it to |0⟩ (can be recycled for a future `*` operation);"""
        self._c_mult_x_mod_n_helper(
            control_qubit=control_qubit,
            qubits=qubits,
            auxilliary_qubits_overflow_extension=auxilliary_qubits_overflow_extension,
            x=x_inverse, n=n,
            ancilla_qubit=ancilla_qubit,
            reverse=True,
        )

        return self

    # -----------------------------------------------------------------------

    def quantum_fourier_transform(self, qubits):
        """
        Quantum Fourier Transform.
        :param qubits: Qubits on which we want to apply the QFT.
        :type qubits: list[Qubit] or QuantumRegister
        :return: self
        """
        n_qubits = len(qubits)
        """Apply HADs and CPHASEs:"""
        for qft_block_idx in range(n_qubits - 1, -1, -1):
            self.circuit.h(qubits[qft_block_idx])
            for j in range(qft_block_idx - 1, -1, -1):
                theta = - pi / 2 ** (qft_block_idx - j)  # Beware: rotation angle theta must be negative!
                self.c_phase(
                    theta=theta,
                    control_qubit=qubits[j],
                    target_qubit=qubits[qft_block_idx]
                )
        """Apply SWAPs:"""
        for k in range(n_qubits // 2):
            self.swap(qubits[k], qubits[n_qubits - k - 1])
        return self

    def inverse_quantum_fourier_transform(self, qubits):
        """
        Inverse Quantum Fourier Transform.
        :param qubits: Qubits on which we want to apply the Inv_QFT.
        :type qubits: list[Qubit] or QuantumRegister
        :return: self
        """
        n_qubits = len(qubits)
        """Apply SWAPs:"""
        for k in range(n_qubits // 2):
            self.swap(qubits[k], qubits[n_qubits - k - 1])
        """Apply HADs and CPHASEs:"""
        for qft_block_idx in range(n_qubits):
            self.circuit.h(qubits[qft_block_idx])
            for j in range(qft_block_idx + 1, n_qubits):
                theta = pi / 2 ** (j - qft_block_idx)
                self.c_phase(
                    theta=theta,
                    target_qubit=qubits[j],
                    control_qubit=qubits[qft_block_idx]
                )
        return self

    # -----------------------------------------------------------------------

    def initialize_qreg(self, qreg: QuantumRegister, init_val: int):
        """
        Initialize the given quantum register with the given value
        :param qreg: the quantum register to initialize
        :param init_val: the value to initialize with
        :return: self
        """
        assert init_val < 2 ** len(qreg), 'Value is too large for register size'
        init_val_bitstring = format(init_val, f'0{len(qreg)}b')
        init_val_statevector = Statevector.from_label(init_val_bitstring)
        self.circuit.initialize(init_val_statevector, qreg)
        return self

    def measure_qreg_to_creg(self, qreg: QuantumRegister, creg: ClassicalRegister):
        """
        Measure the given quantum register and store the measurements in the given classical register
        :param qreg: the quantum register
        :param creg: the classical register
        :return: self
        """
        self.circuit.measure(qreg, creg)
        return self

    # -----------------------------------------------------------------------

    def get_shor_repeat_period(self) -> int:
        """
        Get the repeat period determined by this instance of Shor's factorization algorithm
        :return: the repeat period (i.e., the order of coprime w.r.t. the number to factorize)
        """
        return len(self.result.get_counts(self.circuit))

In [50]:
"""Run Shor's algorithm here:"""

"""Choose number to factorize:"""
N = 15

"""Choose number of precision bits:"""
N_PRECISION_BITS = 4

"""Compute two number factorization of N:"""
factors = ShorQuantumCircuit.shor_algorithm(n=N, n_precision_bits=N_PRECISION_BITS, n_shots=20)
print(f'Factorization of N: {N} = {factors[0]} x {factors[1]}')

Factorization of N: 15 = 3 x 5
